# 使用WAM预测剪接位点

## 模型原理


## 模型实现

### 依赖加载与数据挂载
在本次上机中，本人使用Python（Jupyter Notebook）实现WAM模型。代码在Google Colab的云服务上运行，数据存储在账号对应的Google Drive上，故需要执行额外的代码，进行云盘的挂载。

调用的依赖库中，``os``用于读取文件，``re``用于正则表达式辨别数据中的外显子信息，``matplotlib``用于绘制图像，``tqdm``用于生成数据读取的进度条，``sklearn``用于提供现成的模型性能评估方法。



In [ ]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc, f1_score, confusion_matrix

from google.colab import drive
drive.mount('/content/drive')

training_path = '/content/drive/MyDrive/Collab Files/donor_dataset/testing'
testing_path = '/content/drive/MyDrive/Collab Files/donor_dataset/testing'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 处理训练数据
#### 前景与背景碱基频数的统计
函数``nucleotide_frequency_count``进行碱基前景频率和背景频率的统计：逐文件读取训练用的序列数据，并通过正则表达式读取外显子信息，识别其中的donor位点。在识别到位点后，将位点与位点前``k1``个碱基、位点后``k2``个碱基的片段提取出来，用于前景频率的统计。同时对整个序列中的碱基进行统计，得到背景频率。

函数返回的结果为两个numpy数组，形状分别为``(k1+k2+1, 4, 4)``和``(4, 4)``，对应统计的前景和背景频率。判别用的小片段的首位不存在“前一位碱基”，只需统计该位上的碱基频率，约定储存在``(0, 0, x)``位置（``x``为碱基对应的下标）。

四个碱基和数组下标的对应关系由字典``nucleotide_to_num``提供。提供的序列数据中，除``a``、``t``、``c``、``g``四个碱基以外，还包含表示不确定碱基的``n``、``b``等。在统计时，不确定的碱基位点将会被直接忽略。

In [ ]:
# nucleotide_to_num：提供碱基与数组下标的映射关系
nucleotide_to_num = {
    'a': 0,
    't': 1,
    'c': 2,
    'g': 3
}

# is_nt：判断输入是否为a、t、c、g之一
def is_nt(nt):
    return nt in 'atcg'

print('nucleotide_to_num: ', nucleotide_to_num)

nucleotide_to_num:  {'a': 0, 't': 1, 'c': 2, 'g': 3}


In [ ]:
# nucleotide_frequency_count：进行碱基前景频率和背景频率的统计

def nucleotide_frequency_count(file_path, k1=4, k2=4, nucleotide_to_num=nucleotide_to_num):
    foreground_freq = np.zeros((k1 + k2 + 1, 4, 4), dtype=np.int)
    background_freq = np.zeros((4, 4), dtype=np.int)

    # 逐个读取文件，进行统计
    for data_file in tqdm(os.listdir(file_path)):
        with open(file_path + '/' + data_file, 'r') as FILE:
            lines = FILE.readlines()
            sequence = ''.join(lines[2:]).replace('\n', '').lower()
            donor_sites = re.findall('(\d+)(?=,)', lines[1])

            # 逐位点进行前景频率的统计
            for site in donor_sites:
                # 提取判别用的片段
                site_num = eval(site)
                subsequence = sequence[site_num - k1 - 1:site_num + k2]
                
                # 特别处理第一位
                if is_nt(subsequence[0]):
                    current_nt = nucleotide_to_num[subsequence[0]]
                    foreground_freq[0, 0, current_nt] += 1

                # 统计剩余的每一位
                for i in range(1, k1 + k2 + 1):
                    if not is_nt(subsequence[i]) or not is_nt(subsequence[i - 1]):
                        continue
                    current_nt = nucleotide_to_num[subsequence[i]]
                    previous_nt = nucleotide_to_num[subsequence[i - 1]]
                    foreground_freq[i, previous_nt, current_nt] += 1

            # 统计背景频率
            for i in range(1, len(sequence)):
                if not is_nt(sequence[i]) or not is_nt(sequence[i - 1]):
                    continue
                current_nt = nucleotide_to_num[sequence[i]]
                previous_nt = nucleotide_to_num[sequence[i - 1]]
                background_freq[previous_nt, current_nt] += 1

    return foreground_freq, background_freq

In [ ]:
foreground_freq, background_freq = nucleotide_frequency_count(training_path)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note lin